In [ ]:
# Installs Dependencies
#!pip3 install nltk

In [ ]:
# Imports Dependencies
#import nltk
import copy
import numpy as np
import os
import pandas as pd
import sys

from nltk.corpus import wordnet as wn

from sys import getrecursionlimit

In [ ]:
# Displays Installed PIP Packages
#!pip3 list

In [ ]:
# Downloads Desired NLTK Resources
#nltk.download()

In [ ]:
# Determines & Sets Recusion Limits
print(getrecursionlimit())
sys.setrecursionlimit(4000)
print(getrecursionlimit())

In [ ]:
def determine_base_word_synonyms(base_word):
    base_word_synonyms = set()
    
    for synset in wn.synsets(base_word):
        for lemma in synset.lemmas():
            base_word_synonyms.add(lemma.name().lower())
            
    #print(f'Word: {base_word} - Synonyms: {base_word_synonyms}')

    return base_word_synonyms

In [ ]:
def recursively_determine_synonyms(base_word, base_word_synonyms):
    recursive_synonyms = set()
    recursive_synonyms.update(base_word_synonyms)
    
    for index in range(len(base_word_synonyms)):
        if sorted(base_word_synonyms)[index] != base_word:
            recursive_synonyms.update(determine_base_word_synonyms(sorted(base_word_synonyms)[index]))

    return recursive_synonyms

In [ ]:
def determine_all_synonyms(base_word):
    starting_synonyms = determine_base_word_synonyms(base_word)
    ending_synonyms = recursively_determine_synonyms(base_word, starting_synonyms)
    return ending_synonyms

In [ ]:
def generate_requirements_dictionary(requirements_lists):
    requirements_list = []
    
    for requirements_sublist in requirements_lists:
        for requirement_indicator in requirements_sublist:
            requirements_list.append(requirement_indicator)
    
    dictionary_string = "{'Requirements_Text_Description': [], 'Include': []"
    
    for index in range(len(requirements_list)):     
        dictionary_string += ", '{}': []".format(requirements_list[index])
        
        if index == max(range(len(requirements_list))):
            dictionary_string += '}'
    
    requirements_dictionary = eval(dictionary_string)
    
    return requirements_dictionary

In [ ]:
def determine_inclusive_terms(inclusion_exclusion_terms_lists):
    inclusion_terms_synonyms = set()
    inclusion_exclusion_recursive_terms_synonyms = set()
    
    for term_list in inclusion_exclusion_terms_lists:
        if inclusion_exclusion_terms_lists.index(term_list) == 0:
            inclusion_term_set = set(term_list)
            for inclusion_term in list(inclusion_term_set):
                inclusion_terms_synonyms.update(determine_base_word_synonyms(inclusion_term))
                inclusion_exclusion_recursive_terms_synonyms.update(determine_all_synonyms(inclusion_term))
        else:
            exclusion_term_set = set(term_list)
            for exclusion_term in list(exclusion_term_set):
                inclusion_exclusion_recursive_terms_synonyms.update(determine_all_synonyms(exclusion_term))
                
    return inclusion_terms_synonyms, inclusion_exclusion_recursive_terms_synonyms

In [ ]:
def generate_requirement_synonyms_dictionary(requirements_list):
    dictionary_string = "{"
    
    for index in range(len(requirements_list)):     
        dictionary_string += "'{}': {}".format(requirements_list[index], 
                                               list(determine_base_word_synonyms(requirements_list[index])))
        
        if index != max(range(len(requirements_list))):
            dictionary_string += ', '
        else:
            dictionary_string += '}'
    
    requirement_terms_synonyms_dictionary = eval(dictionary_string)
    
    return requirement_terms_synonyms_dictionary

In [ ]:
def generate_requirement_score_dictionary(req_dict):
    req_scrs_dict = copy.deepcopy(req_dict)
    del req_scrs_dict['Requirements_Text_Description']
    del req_scrs_dict['Include']
    return req_scrs_dict

In [ ]:
def score_inclusivity(term, inclusive_terms):
    
    if term in inclusive_terms:
        return 1
    else:
        return 0

In [ ]:
def score_requirement():
    

In [ ]:
def append_attribute(attributes, depth):
    result = []
    
    if(depth == len(attributes) - 1):
        for i in attributes[depth]:
            result.append(i)

    else:
        sub = append_attribute(attributes, depth + 1)
        for i in attributes[depth]:
            for j in sub:
                result.append(i + ' ' + j)

    return result

In [ ]:
def append_attribute2(attributes, synonyms, depth, scr_dict):
    result = []
    
    count = 0
    
    if(depth == len(attributes) - 1):
        #print(attributes[depth])
        for index in range(len(attributes[depth])):
            for synonym in synonyms[depth].get(attributes[depth][index]):
                
                #if count < 1:
                #    print(synonyms[depth])
                #    print(attributes[depth][index])
                #    print(attributes[depth])
                #    count += 1
                
                result.append(synonym)
                
                #for attribute in attributes[depth]:
                #    if attribute == attributes[depth][index]:
                #        scr_dict[attribute].append(1)
                #    else:
                #        scr_dict[attribute].append(0)
                
    else:
        sub = append_attribute2(attributes, synonyms, depth + 1, scr_dict)
        for index in range(len(attributes[depth])):
            for index2 in range(len(sub)):
                #result.append(attributes[depth][index] + ' ' + sub[index2])
                for synonym in synonyms[depth].get(attributes[depth][index]):
                    result.append(synonym + ' and ' + sub[index2])
                    
                    for attribute in attributes[depth]:
                        if attribute == attributes[depth][index]:
                            scr_dict[attribute].append(1)
                        else:
                            scr_dict[attribute].append(0)
                
    return result



In [ ]:
def append_attribute3(attributes, synonyms, depth):
    result = []
    
    count = 0
    
    if(depth == len(attributes) - 1):
        #print(attributes[depth])
        
        for index in range(len(attributes[depth])):
            attribute = attributes[depth][index]
            for synonym in synonyms[depth].get(attribute):
                
                zeros = [0] * len(attributes[depth])
                zeros[index] = 1
                
                result.append((synonym, zeros))
    else:
        sub = append_attribute3(attributes, synonyms, depth + 1)
        for index in range(len(attributes[depth])):
            for index2 in range(len(sub)):
                for synonym in synonyms[depth].get(attributes[depth][index]):
                    
                    zeros = [0] * len(attributes[depth])
                    zeros[index] = 1
                
                    result.append((synonym + ' and ' + sub[index2][0], zeros + sub[index2][1]))
                
    return result




In [ ]:
def generate_requirement_text_combinations(inclusivity_terms, attributes):
    requirement_text_combinations = []
    
    for i in inclusivity_terms:
        result = append_attribute(attributes, 0)
        
        for j in result:
            requirement_text_combinations.append(i + ' ' + j)
            
    return requirement_text_combinations

In [ ]:
def generate_requirement_text_combinations2(inclusive_terms, 
                                            inclusivity_terms, 
                                            attributes, 
                                            synonyms, 
                                            scr_dict):
    
    requirement_text_combinations = []
    inclusion_scores = []
    
    for i in inclusivity_terms:
        result = append_attribute2(attributes, synonyms, 0, scr_dict)
        
        for j in result:
            requirement_text_combinations.append(i + ' ' + j)
            inclusion_scores.append(score_inclusivity(i, inclusive_terms))
            
    return requirement_text_combinations, inclusion_scores

In [ ]:
def generate_requirement_text_combinations3(inclusive_terms, 
                                            inclusivity_terms, 
                                            attributes, 
                                            synonyms):
    aslist = []
    for i in inclusivity_terms:
        result = append_attribute3(attributes, synonyms, 0)
        
        for j in result:
            requirement = i + " " + j[0]
            scores = j[1]
            scores.append(score_inclusivity(i, inclusive_terms))
            aslist.append([requirement] + scores)
            #aslist.append([requirement] + j[1])
            
    return aslist

In [ ]:
def generate_synthetic_requirements_dataset(texts, 
                                            inclusion_scores, 
                                            scr_dict,
                                            dictionary):
    dataframe = pd.DataFrame.from_dict(dictionary)
    print(dataframe)
    print(len(texts))
    print(len(inclusion_scores))
    dataframe['Requirements_Text_Description'] = np.array(texts)
    dataframe['Include'] = np.array(inclusion_scores)
    
    for i in range(3):
        key = list(scr_dict)[i]
        print(len(scr_dict[key]))
        dataframe[key] = np.array(scr_dict[key])
    
    #for key in scr_dict:
    #    dataframe[key] = np.array(scr_dict[key])
    
    return dataframe

In [ ]:
def generate_synthetic_requirements_dataset2(texts_scores_list, requirements):
    
    requirements_list = ['description']
    for sublist in requirements:
        for requirement in sublist:
            requirements_list.append(requirement)
    requirements_list.append('inclusion')
    
    dataframe = pd.DataFrame(texts_scores_list, 
                             columns = requirements_list)
    return dataframe

In [ ]:
def main():
    req_lst = [['red', 'green', 'blue'], ['triangle', 'square']]
    incl_trm_lst = [['include'], ['exclude']]
    
    req_dict = generate_requirements_dictionary(req_lst)
    req_scrs_dict = generate_requirement_score_dictionary(req_dict)
    
    incl_trms, nincl_trms = determine_inclusive_terms(incl_trm_lst)
    
    syns = copy.deepcopy(req_lst)
    
    for index in range(len(syns)):
        syns[index] = (generate_requirement_synonyms_dictionary(syns[index]))
        
    req_txts, incls_scrs = generate_requirement_text_combinations2(incl_trms, 
                                                                   nincl_trms, 
                                                                   req_lst, 
                                                                   syns, 
                                                                   req_scrs_dict)
    
    req_df = generate_synthetic_requirements_dataset(req_txts, 
                                                     incls_scrs, 
                                                     req_scrs_dict,
                                                     req_dict)
    
    data_file_path = '../../Data/'
    req_df.to_csv(os.path.join(data_file_path, 'synthetic_requirements_data.csv'))
    
    #req_df.describe()
    #req_df.head()
    #req_df.tail()
    display(req_df)

In [ ]:
main()

In [ ]:
word_to_find_synonyms_for = 'green'

In [ ]:
base_synonyms = determine_base_word_synonyms(word_to_find_synonyms_for)
recursive_synonyms = recursively_determine_synonyms(word_to_find_synonyms_for, base_synonyms)
all_synonyms = determine_all_synonyms(word_to_find_synonyms_for)

In [ ]:
print(base_synonyms)

In [ ]:
print(recursive_synonyms)

In [ ]:
print(all_synonyms)

In [ ]:
lists_of_requirements_for_dataframe_columns = [['red', 'green', 'blue'], ['triangle', 'square']]

In [ ]:
print(lists_of_requirements_for_dataframe_columns)

In [ ]:
print(len(lists_of_requirements_for_dataframe_columns))

In [ ]:
requirements_dictionary = generate_requirements_dictionary(lists_of_requirements_for_dataframe_columns)

In [ ]:
print(requirements_dictionary)

In [ ]:
req_scrs_dict = generate_requirement_score_dictionary(requirements_dictionary)

In [ ]:
print(req_scrs_dict)

In [ ]:
print(req_scrs_dict.keys())

In [ ]:
inclusion_exclusion_terms_lists = [['include'], ['exclude']]

In [ ]:
terms_of_inclusion, nonviable_terms_of_inclusion_exclusion = (
    determine_inclusive_terms(inclusion_exclusion_terms_lists))

In [ ]:
print(terms_of_inclusion)

In [ ]:
print(nonviable_terms_of_inclusion_exclusion)

In [ ]:
synonyms = lists_of_requirements_for_dataframe_columns.copy()

In [ ]:
print(synonyms)

In [ ]:
for index in range(len(synonyms)):
    synonyms[index] = (generate_requirement_synonyms_dictionary(synonyms[index]))

In [ ]:
print(synonyms)

In [ ]:
list(synonyms[0].keys())

In [ ]:
print(lists_of_requirements_for_dataframe_columns)

In [ ]:
result = append_attribute(lists_of_requirements_for_dataframe_columns, 0)
#result = append_attribute(requirement_terms_synonyms_dictionaries, 0)

In [ ]:
print(result)

In [ ]:
result2 = append_attribute2(lists_of_requirements_for_dataframe_columns, 
                            synonyms, 
                            0, 
                            req_scrs_dict)

In [ ]:
result3 = append_attribute3(lists_of_requirements_for_dataframe_columns, 
                            synonyms, 
                            0)

In [ ]:
print(req_scrs_dict)

In [ ]:
print(len(result2))
print(len(req_scrs_dict['red']))
print(len(req_scrs_dict['triangle']))

In [ ]:
print(len(nonviable_terms_of_inclusion_exclusion))

In [ ]:
print(3822 * 122)

In [ ]:
requirement_texts = generate_requirement_text_combinations(nonviable_terms_of_inclusion_exclusion, 
                                                           lists_of_requirements_for_dataframe_columns)

In [ ]:
requirement_texts2, inclusion_scores = generate_requirement_text_combinations2(
    terms_of_inclusion, 
    nonviable_terms_of_inclusion_exclusion, 
    lists_of_requirements_for_dataframe_columns, 
    synonyms,
    req_scrs_dict)

In [ ]:
requirement_texts_scores = generate_requirement_text_combinations3(
    terms_of_inclusion,
    nonviable_terms_of_inclusion_exclusion, 
    lists_of_requirements_for_dataframe_columns, 
    synonyms)

In [ ]:
for i in range(10):
    print(requirement_texts_scores[i])

In [ ]:
print(len(requirement_texts_scores))

In [ ]:
for text in requirement_texts:
    print(text)

In [ ]:
requirements_dataframe = generate_synthetic_requirements_dataset(requirement_texts2, 
                                                                 inclusion_scores, 
                                                                 req_scrs_dict,
                                                                 requirements_dictionary)

In [ ]:
requirements_dataframe2 = generate_synthetic_requirements_dataset2(requirement_texts_scores, 
                                                                   lists_of_requirements_for_dataframe_columns)

In [ ]:
requirements_dataframe2.describe()

In [ ]:
requirements_dataframe2.head()

In [ ]:
requirements_dataframe2.tail()

In [ ]:
data_file_path = '../../Data/'

In [ ]:
requirements_dataframe2.to_csv(os.path.join(data_file_path, 'synthetic_requirements_data.csv'))